# Chatbot Test Notebook

This notebook is designed to test and experiment with the functionality of the chatbot implemented in the `rag_swimrules` project. It showcases various scenarios and evaluates the performance of the chatbot.

In [1]:
import textwrap

from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from langchain_chroma.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


In [2]:


swimming_rules_template = """You are an expert swimming rules assistant trained on FINA and competitive swimming regulations. 
Your answers must be precise, factual, and based only on the provided rules. Follow these guidelines:

1. Answer ONLY using the verified rules provided below
2. DO NOT use information from any other sources, including the internet or personal knowledge.
3. if an action is not explicitly prohibited in the rules, assume it is allowed.
4. if an action is not explicity mentioned, assume it is allowed.
5. if an action is explicitly stated in the rule, assume it is allowed.
6. A disqualification msut be based on a specific rule violation and cannot be implied.
7. If the information isn't in the rules, say "I don't have that rule in my current documentation"
8. Respond with either "LEGAL" or "DISQUALIFIED" based on the swimming rules, provide a brief, concise explanation.
9. Keep answers concise but complete
10. Reference specific rule numbers when possible

Relevant swimming rules:
{context}

Question: {question}

Answer:"""

SWIMMING_RULES_PROMPT = PromptTemplate(
    template=swimming_rules_template,
    input_variables=["context", "question"]
)


In [3]:
queries = [
    "Prior to the start signal the swimmer is seen rocking back and forth.  Is this a disqualification?",
    "upon the command 'take your mark', the swimmer takes their starting position and just prior to the start signal, the swimmer is seen moving forward.",
    "What happens if a swimmer touches the wall with one hand in freestyle?",
    "what happens if a swimmer touches the wall with one hand in breaststroke?",
    "A swimmer is wearing a wetsuit in a swimming competition?",
    "During a freestyle event, a swimmer does a flip turn and touches the wall with only one foot.",
    "A breaststroke swimmer moves their hands in a sculling or flipper movement at the end of the first arm stroke, both after the start and after the turn Should they be disqualified?",
    "In a 9-10 100 yard breaststroke event, a swimmer completes 50 yards with a simulatenous two-hand touch and, thinking that the race is over, pushes back from the wall to read the scoreboard. At this point, realizing that the race is only halfway over, the swimmer returns to the wall, pushes off on the breast, and completes the required distance in good form. What call, if any, should be made?",
    "In a 9-10 100 yard breaststroke event, a swimmer completes 50 yards with a simulatenous two-hand touch and, thinking that the race is over, pushes back from the wall on their back to read the scoreboard. At this point, realizing that the race is only halfway over, the swimmer returns to the wall, pushes off on the breast, and completes the required distance in good form. What call, if any, should be made?",
    "A swimmer swims the breaststroke in a way that their hands are completely underwater when they are pushed forward together from their breast.",
    "A swimmer swims the breaststroke in a way that their elbows are above the water line during the forward movement of the arms.",
    "A breaststroker’s head breaks the surface of the water during each cycle, however, the swimmer does not take a breath even if the head breaks the surface.",
    "During a freestyle event, a swimmer starts in the water such that the swimmwer is facing the starting end of the pool with one hand on the wall.",
    "During a freestyle event, a swimmer starts in the water such that the swimmer is facing the other end of the pool with one hand on the wall.",
    "During a butterfly event, a swimmer starts in the water such that the swimmer is facing the other end of the pool with one hand on the wall.",
    "During a butterfly event, a swimmer starts in the water such that the swimmer has one hand on the wall looking away from the starting block.",
    "During a butterfly event, a swimmer starts in the water such that the swimmer is looking at the opposite end of the pool.",
    "In a 200m individual medley event, a swimmer swims the first 50m in butterfly, the second 50m in backstroke, the third 50m starts swimming freestyle but then switches to breaststroke, and the last 50m in freestyle.",
    "In a 100y freestyle event, the swimmer swims the first 50y in butterfly and the last 50y in backstroke.",
]

In [21]:
# Initialize embeddings model (ensure the same model used during creation)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# retretieve the relevant rules from your database or knowledge base
# Load the existing Chroma vector store
persist_directory = "../agent_swimrules_db"
vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

# Create a retriever from the vector store
retriever = vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 20, "score_threshold": 0.4})



In [22]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=1000
)


In [23]:

for query in queries[:3]:
    print()
    print(textwrap.fill(f"\n>>>Query: {query}", width=80))
    

    # Use the retriever to get relevant documents
    docs = retriever.get_relevant_documents(query)
    print(f">>>Retrieved {len(docs)} documents")
    for doc in docs:
        print(doc.metadata)
        # print(textwrap.fill(f"\n>>>Retrieved Document:\n{doc.page_content}", width=80)) 
    # Combine the retrieved documents into a context string
    context = "\n".join([doc.page_content for doc in docs])
    # print(textwrap.fill(f"\n>>>CONTEXT:\n{context}", width=80))
    # If no documents were retrieved, provide a default message

    # Create the prompt template
    prompt = HumanMessage(SWIMMING_RULES_PROMPT.format(context=context, question=query))

    response = llm([prompt])
    # Word wrap the response content if the line is longer than 80 characters
    wrapped_response = textwrap.fill(response.content, width=80)
    print(f">>>Assessment: {wrapped_response}")



 >>>Query: Prior to the start signal the swimmer is seen rocking back and forth.
Is this a disqualification?
>>>Retrieved 8 documents
{'chunk_type': 'guidance', 'rule_title': 'Interpretation Guidance', 'rule_number': 'GUIDANCE', 'page': '0', 'source': './rule_data/interpretation_guidance.txt', 'category': 'Interpretations', 'rule_id': 'guidance', 'stroke_type': 'none', 'section': 'Guidance'}
{'chunk_type': 'rule', 'section': 'General', 'category': 'General Rules', 'rule_id': 'page_21_chunk_32', 'rule_number': 'unknown', 'page': '21', 'source': './rule_data/2025-mini-rulebook.pdf', 'rule_title': 'General Content', 'stroke_type': 'none'}
{'source': './rule_data/2025-mini-rulebook.pdf', 'rule_title': 'A disqualification can be made only by the official within whose jurisdiction the', 'rule_number': '102.1', 'page': '43', 'category': 'Officials', 'section': 'DISQUALIFICATIONS', 'rule_id': 'rule_102.1', 'stroke_type': 'none', 'chunk_type': 'rule'}
{'rule_title': 'General Content', 'section